In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim

In [2]:
# Download the dataset
mnist = tf.keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.
print(f"X_train.shape => {X_train.shape}")
print(f"X_test.shape => {X_test.shape}")
print(f"Y_train.shape => {Y_train.shape}")
print(f"Y_test.shape => {Y_test.shape}")

X_train.shape => (60000, 28, 28)
X_test.shape => (10000, 28, 28)
Y_train.shape => (60000,)
Y_test.shape => (10000,)


In [3]:
def build_autoencoder(input_shape, latent_dim, encoder_units, decoder_units, dropout_rate):
    # Encoder
    encoder_input = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Flatten()(encoder_input)
    for units in encoder_units:
        x = tf.keras.layers.Dense(units, activation='relu')(x)
        x = tf.keras.layers.Dropout(dropout_rate)(x)
    encoded_output = tf.keras.layers.Dense(latent_dim, activation='relu')(x)
    encoder = tf.keras.Model(inputs=encoder_input, outputs=encoded_output, name='encoder')

    # Decoder
    decoder_input = tf.keras.Input(shape=(latent_dim,))
    x = decoder_input
    for units in decoder_units:
        x = tf.keras.layers.Dense(units, activation='relu')(x)
        x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.Dense(tf.math.reduce_prod(input_shape), activation='relu')(x)
    decoded_output = tf.keras.layers.Reshape(input_shape)(x)
    decoder = tf.keras.Model(inputs=decoder_input, outputs=decoded_output, name='decoder')

    # Autoencoder
    autoencoder_input = tf.keras.Input(shape=input_shape)
    encoded = encoder(autoencoder_input)
    decoded = decoder(encoded)
    autoencoder = tf.keras.Model(inputs=autoencoder_input, outputs=decoded, name='autoencoder')

    return autoencoder, encoder, decoder


In [4]:
def calculate_psnr(original, reconstructed):
    return compare_psnr(original, reconstructed, data_range=original.max() - original.min())

def calculate_ssim(original, reconstructed):
    return compare_ssim(original, reconstructed, multichannel=True)


In [5]:
def train_model_with_early_stopping(model, X_train, epochs, batch_size, patience=5):
    """
    Trains the given model on the provided data with early stopping.

    Parameters:
    model: A TensorFlow/Keras model.
    X_train (np.array): Training data.
    epochs (int): Number of epochs to train the model.
    batch_size (int): Batch size for training.
    patience (int): Number of epochs to wait for improvement before stopping.

    Returns:
    history: Training history object.
    """
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=patience, restore_best_weights=True)

    history = model.fit(X_train, X_train, epochs=epochs, batch_size=batch_size,
                        validation_split=0.2, callbacks=[early_stopping])
    return history

In [6]:
def predict_model(model, X_test):
    predictions = model.predict(X_test)
    return predictions


def plot_training_history(history):
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss During Training')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid(True)
    plt.show()

In [7]:
# Define model parameters
input_shape = X_train.shape[1:]
dropout_rate = 0.2
output_shape = X_train.shape[1:]
encoder_units = [512, 256, 128]
decoder_units = [128, 256, 512]

In [8]:
latent_dims = [2, 8, 32, 64]
autoencoders = dict()

for latent_dim in latent_dims:
    # Build and compile the autoencoder
    autoencoder, _, _ = build_autoencoder(input_shape, latent_dim, encoder_units, decoder_units, dropout_rate)
    autoencoder.compile(loss='mae', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
    autoencoders[latent_dim] = autoencoder
    print(f"Latent dim => {latent_dim}")
    print(autoencoder.summary(expand_nested=True))



Latent dim => 2
Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 encoder (Functional)        (None, 2)                 566402    
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| input_1 (InputLayer)       [(None, 28, 28)]          0        |
|                                                               |
| flatten (Flatten)          (None, 784)               0        |
|                                                               |
| dense (Dense)              (None, 512)               401920   |
|                                                               |
| dropout (Dropout)          (None, 512)               0        |
|                                                               |
| dense_1 (Dense)            (None, 25

In [11]:
def visualize_reconstructions(original, reconstructed, n_images, title):
    plt.figure(figsize=(20, 4))
    for i in range(n_images):
        # Display original image
        ax = plt.subplot(2, n_images, i + 1)
        plt.imshow(original[i].reshape(28, 28), cmap='gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # Display reconstructed image
        ax = plt.subplot(2, n_images, i + 1 + n_images)
        plt.imshow(reconstructed[i].reshape(28, 28), cmap='gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    plt.suptitle(title)
    plt.show()

def train_model(model, X_train, epochs, batch_size, patience=5):
    """
    Trains the given model on the provided data with early stopping.

    Parameters:
    model: A TensorFlow/Keras model.
    X_train (np.array): Training data.
    epochs (int): Number of epochs to train the model.
    batch_size (int): Batch size for training.
    patience (int): Number of epochs to wait for improvement before stopping.

    Returns:
    history: Training history object.
    """
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=patience, restore_best_weights=True)

    history = model.fit(X_train, X_train, epochs=epochs, batch_size=batch_size,
                        validation_split=0.2, callbacks=[early_stopping])
    return history

In [ ]:
reconstructed_images = dict()
psnr_values = []
ssim_values = []

for latent_dim, autoencoder in autoencoders.items():
    print(f"Latent dim => {latent_dim}")
    # Train the model
    history = train_model(autoencoder, X_train, epochs=50, batch_size=128)
    # Predict using the model
    predictions = predict_model(autoencoder, X_test)
    reconstructed_images[latent_dim] = predictions
    # Plot the training history
    plot_training_history(history)

    # Calculate PSNR and SSIM
    psnr = calculate_psnr(X_test, predictions)
    ssim = calculate_ssim(X_test, predictions)
    psnr_values.append(psnr)
    ssim_values.append(ssim)

    # Visualization
    title = f"Autoencoder with Latent Dimension {latent_dim} - PSNR: {psnr:.2f}, SSIM: {ssim:.2f}"
    visualize_reconstructions(X_test, predictions, n_images_to_display, title)


In [ ]:
# Select an image for comparison
index_of_image_to_compare = 0  # You can change this index
selected_image = X_test[index_of_image_to_compare]

plt.figure(figsize=(20, 10))
for i, latent_dim in enumerate(latent_dims):
    reconstructed_image = reconstructed_images[latent_dim][index_of_image_to_compare]
    
    # Display reconstructed image
    ax = plt.subplot(2, len(latent_dims), i + 1)
    plt.imshow(reconstructed_image.reshape(28, 28), cmap='gray')
    plt.title(f"Latent Dim: {latent_dim}\nPSNR: {psnr_values[i]:.2f}\nSSIM: {ssim_values[i]:.2f}")
    plt.axis('off')

# Display the original image for comparison
ax = plt.subplot(2, len(latent_dims), len(latent_dims) + 1)
plt.imshow(selected_image.reshape(28, 28), cmap='gray')
plt.title("Original Image")
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Plot the PSNR and SSIM graphs
fig, ax1 = plt.subplots(figsize=(8, 4))

# PSNR Plot
color = 'tab:red'
ax1.set_xlabel('Latent Dimension')
ax1.set_ylabel('PSNR (dB)', color=color)
ax1.plot(latent_dims, psnr_values, 'o-', color=color)
ax1.tick_params(axis='y', labelcolor=color)

# SSIM Plot on the same x-axis
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
color = 'tab:blue'
ax2.set_ylabel('SSIM', color=color)
ax2.plot(latent_dims, ssim_values, 's-', color=color)
ax2.tick_params(axis='y', labelcolor=color)

# Layout and grid
fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.grid(True)
plt.title('PSNR and SSIM Values for Different Latent Dimensions')

plt.show()